In [1]:
import os

In [2]:
%pwd

'e:\\Udemy Resources\\MLOps Krish Naik\\Wine-quality-prediction-mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Udemy Resources\\MLOps Krish Naik\\Wine-quality-prediction-mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.wine_quality_predictor.constants import *
from src.wine_quality_predictor.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath: Path = CONFIG_FILE_PATH, 
                 params_filepath: Path = PARAMS_FILE_PATH,
                 schema_filepath: Path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
from src.wine_quality_predictor import logger
import zipfile

In [13]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-18 15:57:24,313: INFO: common: YAML file: config\config.yaml loaded successfully:]
[2025-11-18 15:57:24,327: INFO: common: YAML file: params.yaml loaded successfully:]
[2025-11-18 15:57:24,329: INFO: common: YAML file: schema.yaml loaded successfully:]
[2025-11-18 15:57:24,331: INFO: common: Created directory at: artifacts:]
[2025-11-18 15:57:24,333: INFO: common: Created directory at: artifacts\data_ingestion:]
[2025-11-18 15:57:26,062: INFO: 251335794: artifacts\data_ingestion\data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4355:297D95:20E9AD:40D23C:691C4A0B
Accept-Ranges: bytes
Date: Tue